In [24]:
!~/anaconda3/bin/pip install beautifulsoup4 lxml

In [95]:
import time
import requests
from bs4 import BeautifulSoup as bs

In [96]:
class RecipeData:
    """Data class representing recipe
    """
    
    def __init__(self, **kwargs):
        self.name: str = kwargs.get('name')
        self.ingredients: set = kwargs.get('ingredients')
        self.url: str = kwargs.get('url')
        self.views: int = kwargs.get('views')
        self.comments: int = kwargs.get('comments')
        self.likes: int = kwargs.get('likes')
        self.category: int = kwargs.get('category')
            
    def __hash__(self) -> int:
        return hash(frozenset(self.__dict__.items()))
         
    def __eq__(self, other):
        return (
            self.__class__ == other.__class__ and
            self.__dict__ == other.__dict__
        )
        
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}{self.__dict__}'

In [103]:
class PovarenokScrapper:
    
    def __init__(self, category: str, crawl_delay: int = 1):
        self._host: str = 'https://www.povarenok.ru/recipes'
        self._category: str = {
            'it': 'kitchen/56',
            'fr': 'kitchen/64',
            'ch': 'kitchen/73',
            'jp': 'kitchen/79',
            'ru': 'kitchen/101',
            'ua': 'kitchen/104',
        }[category]
        self._crawl_delay: int = crawl_delay
        self._current_page_num: int = 0
        self._last_recipes: set = set()
        self._data_to_return: set = set()

    def __iter__(self) -> iter:
        return self

    def __next__(self):
        if len(self._data_to_return) == 0:
            page = self._next_page()
            recipes = self._extract_recipes(page)
            if self._last_recipes != recipes:
                self._last_recipes = recipes
                self._data_to_return = self._last_recipes.copy()
                time.sleep(self._crawl_delay)
                return self.__next__()
            else:
                raise StopIteration
        else:
            return self._data_to_return.pop()

    def __repr__(self) -> str:
        return '{clazz}({sep}host={host},{sep}category={category},{sep}crawl_delay={crawl_delay},\n)'.format(
            sep='\n\t',
            clazz=self.__class__.__name__,
            host=self._host,
            category=self._category,
            crawl_delay=self._crawl_delay,
        )

    def _get_page(self, page_num: int) -> bs:
        url = f'{self._host}/{self._category}/~{page_num}/'
        content = requests.get(url).text
        parsed = bs(content, 'html.parser')
        return parsed
    
    def _next_page(self) -> bs:
        if self._current_page_num is None:
            self._current_page_num = 0
        self._current_page_num += 1
        return self._get_page(self._current_page_num)
    
    def _extract_recipes(self, page: bs) -> set:
        recipes = set()
        for article in page.find_all('article'):
            name = article.select_one('article > h2').get_text().strip()
            url = article.select_one('article > h2 > a')['href']
            ingr = article.select_one('div.ingr_fast').get_text().strip().lower()
            views = article.select_one('ul.icons-wrap .i-views').get_text()
            comm = article.select_one('ul.icons-wrap .i-comments').get_text()
            likes = article.select_one('ul.icons-wrap .i-likes').get_text()
            category = ' > '.join(map(lambda s: s.get_text().replace('  ', ' '), article.select('div.article-breadcrumbs > p > span')))
            recipe = RecipeData(name=name, url=url, ingredients=ingr, views=views, comments=comm, likes=likes, category=category)
            recipes.add(recipe)
        return recipes
    
    def reset(self):
        self._current_page_num = 0
        self._last_recipes = set()

In [104]:
scrapper = PovarenokScrapper('ch')
scrapper

PovarenokScrapper(
	host=https://www.povarenok.ru/recipes,
	category=kitchen/73,
	crawl_delay=1,
)